# ETL Pipeline
To load data from SQL into python

In [5]:
!pip install sqlalchemy
!pip install dask
!pip install pyodbc

     ---------------------------------------- 74.9/74.9 kB 1.0 MB/s eta 0:00:00


In [3]:
from sqlalchemy import create_engine
import pandas as pd
import dask.dataframe as dd

# Extract
Load all 3 tables from SQL into pandas DataFrames

In [7]:
%%time

# set to your own desktop name
pc_name = 'DESKTOP-9552JHN'

# connect to MS-SQL
server = f"{pc_name}\SQLEXPRESS" # SQL Server Name
database = "JustTaxi" # database name
con_string = f'mssql+pyodbc://{server}/{database}?driver=SQL Server'
engine = create_engine(con_string)

# retrieve data
connection = engine.connect()

# driver data
drivers = connection.execute('SELECT * FROM drivers')
driver_data = pd.DataFrame(data=drivers.fetchall(), columns=drivers.keys())

# trip data
trips = connection.execute('SELECT * FROM trips')
trip_data = pd.DataFrame(data=trips.fetchall(), columns=trips.keys())

connection.close() # close connection explicitly

14.87s - Error inserting pydevd breaks.
Traceback (most recent call last):
  File "c:\Users\Timothy Chia\AppData\Local\Programs\Python\Python310\lib\site-packages\debugpy\_vendored\pydevd\_pydevd_frame_eval\pydevd_modify_bytecode.py", line 328, in insert_pydevd_breaks
    for new_instruction in get_instructions_to_add(
  File "c:\Users\Timothy Chia\AppData\Local\Programs\Python\Python310\lib\site-packages\debugpy\_vendored\pydevd\_pydevd_frame_eval\pydevd_modify_bytecode.py", line 102, in get_instructions_to_add
    Instr("LOAD_CONST", _pydev_stop_at_break, lineno=stop_at_line - 1),
  File "c:\Users\Timothy Chia\AppData\Local\Programs\Python\Python310\lib\site-packages\debugpy\_vendored\pydevd\_pydevd_frame_eval\vendored\bytecode\instr.py", line 171, in __init__
    self._set(name, arg, lineno)
  File "c:\Users\Timothy Chia\AppData\Local\Programs\Python\Python310\lib\site-packages\debugpy\_vendored\pydevd\_pydevd_frame_eval\vendored\bytecode\instr.py", line 239, in _set
    _check_line

CPU times: total: 62.5 ms
Wall time: 70.5 ms


In [8]:
%%time
# get sensor data by chunksize
# connection = engine.connect().execution_options(stream_results=True)
sensor_data_generator = pd.read_sql_query('SELECT * FROM sensor_data', con_string, chunksize=100000)
sensor_data = pd.concat([chunk for chunk in sensor_data_generator])

CPU times: total: 4min 41s
Wall time: 4min 43s


In [9]:
driver_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   driver_id      148 non-null    object
 1   name           148 non-null    object
 2   date_of_birth  148 non-null    object
 3   gender         148 non-null    object
 4   car_model      148 non-null    object
 5   car_make_year  148 non-null    object
 6   rating         148 non-null    object
dtypes: object(7)
memory usage: 8.2+ KB


In [10]:
trip_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   booking_id    20000 non-null  object
 1   driver_id     20000 non-null  object
 2   safety_label  20000 non-null  object
dtypes: object(3)
memory usage: 468.9+ KB


In [11]:
sensor_data.info()  

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7469656 entries, 0 to 69655
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   booking_id      object 
 1   accuracy        float64
 2   bearing         float64
 3   acceleration_x  float64
 4   acceleration_y  float64
 5   acceleration_z  float64
 6   gyro_x          float64
 7   gyro_y          float64
 8   gyro_z          float64
 9   second          float64
 10  speed           float64
dtypes: float64(10), object(1)
memory usage: 683.9+ MB


# Transform
Clean and merge the DataFrames

## Remove duplicated data

In [12]:
# remove duplicated data from all 3 dataframes
driver_data = driver_data.drop_duplicates()
trip_data = trip_data.drop_duplicates()
sensor_data = sensor_data.drop_duplicates()

## Merge dataframes

In [13]:
%%time
# merge driver and trip data
driver_trips = trip_data.merge(driver_data, on='driver_id', how='left')

# merge driver_trips and sensor data
driver_trips_sensor = sensor_data.merge(driver_trips, on='booking_id', how='left')

CPU times: total: 1.67 s
Wall time: 1.68 s


In [14]:
driver_trips_sensor

,booking_id,accuracy,bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,speed,driver_id,safety_label,name,date_of_birth,gender,car_model,car_make_year,rating
0,0,8.000,143.298294,-1.706207,-9.270792,-1.209448,-0.028965,-0.032652,0.015390,2.0,0.228454,48,0,Lilia,1974-08-13,Male,Mercedes-Benz,2013-01-01,4.0
1,0,8.000,143.298294,-1.416705,-9.548032,-1.860977,-0.022413,0.005049,-0.025753,3.0,0.228454,48,0,Lilia,1974-08-13,Male,Mercedes-Benz,2013-01-01,4.0
2,0,8.000,143.298294,-0.346924,-9.532629,-1.204663,0.014962,-0.050033,0.025118,9.0,0.228454,48,0,Lilia,1974-08-13,Male,Mercedes-Benz,2013-01-01,4.0
3,0,8.000,143.298294,-0.600986,-9.452029,-2.157507,0.004548,-0.011713,-0.004078,11.0,0.228454,48,0,Lilia,1974-08-13,Male,Mercedes-Benz,2013-01-01,4.0
4,0,8.000,143.298294,-0.597546,-9.863403,-1.672711,-0.000401,0.000315,-0.009830,12.0,0.228454,48,0,Lilia,1974-08-13,Male,Mercedes-Benz,2013-01-01,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7469651,996432412828,3.712,87.000000,-1.908237,8.988108,2.710319,-0.105388,0.442492,0.058795,1538.0,3.574830,24,1,Sephira,1991-08-27,Male,Chevrolet,2008-01-01,3.5
7469652,996432412828,3.649,64.000000,-0.909824,8.722343,2.320052,-0.120659,0.332536,0.089949,1539.0,3.235918,24,1,Sephira,1991-08-27,Male,Chevrolet,2008-01-01,3.5
7469653,996432412828,3.462,42.000000,-2.490046,9.490906,-0.502798,0.061379,0.327038,0.042912,1540.0,2.388671,24,1,Sephira,1991-08-27,Male,Chevrolet,2008-01-01,3.5
7469654,996432412828,3.620,3.000000,-1.498816,10.146936,-0.668003,0.135293,0.243350,0.093003,1542.0,3.794280,24,1,Sephira,1991-08-27,Male,Chevrolet,2008-01-01,3.5
